# Allgemein

<span style="color:red">A*-Suche auch bidirektional, Iterative Deepening A*</span>

In [1]:
%load_ext memory_profiler

In [2]:
import graphviz as gv
import heapq
from Set import Set

In [3]:
def to_list(State): 
    return [list(row) for row in State]

In [4]:
def to_tuple(State):
    return tuple(tuple(row) for row in State)

# Suchprobleme

## Breitensuche (Breadth First Search)

In [5]:
def rec_path_to(state, Parent):
    p = Parent[state]
    if p == state:
        return [state]
    return rec_path_to(p, Parent) + [state]

In [6]:
def bf_search(start, goal, next_states):
    Frontier = { start }
    Visited  = set()
    Parent   = { start: start }
    while Frontier:
        NewFrontier = set()
        for s in Frontier:
            for ns in next_states(s):
                if ns not in Visited and ns not in Frontier:
                    NewFrontier.add(ns)
                    Parent[ns] = s
                    if ns == goal:
                        print("number of states: ", len(Visited) + len(Frontier) + len(NewFrontier))
                        return rec_path_to(goal, Parent)
        Visited |= Frontier
        Frontier = NewFrontier

## Bidirektionale Breitensuche (Bidirectional Breadth First Search)

In [7]:
def rec_path_to(state, Parent):
    p = Parent[state]
    if p == state:
        return [state]
    return rec_path_to(p, Parent) + [state]

In [8]:
def combinePaths(state, ParentA, ParentB):
        Path1 = rec_path_to(state, ParentA)
        Path2 = rec_path_to(state, ParentB)
        return Path1[:-1] + Path2[::-1] # Path2 is reversed

In [9]:
def bbf_search(start, goal, next_states):        
    FrontierA = { start }
    ParentA   = { start: start }
    FrontierB = { goal }
    ParentB   = { goal: goal } 
    while FrontierA and FrontierB:
        NewFrontier = set()
        for s in FrontierA:
            for ns in next_states(s):
                if ns not in ParentA:
                    NewFrontier |= { ns }
                    ParentA[ns]  = s
                    if ns in ParentB:
                        return combinePaths(ns, ParentA, ParentB)
        FrontierA   = NewFrontier
        NewFrontier = set()
        for s in FrontierB:
            for ns in next_states(s):
                if ns not in ParentB:
                    NewFrontier |= { ns }
                    ParentB[ns]  = s
                    if ns in ParentA:
                        return combinePaths(ns, ParentA, ParentB)
        FrontierB = NewFrontier

## Tiefensuche mit Stack (Depth First Search)

In [10]:
def iter_path_to(state, Parent):
    Path = [state]
    while state != Parent[state]:
        state = Parent[state]
        Path  = [ state ] + Path
    return Path

In [11]:
def df_search(start, goal, next_states):
    Stack  = [start]
    Parent = { start: start }
    while Stack:
        state = Stack.pop()
        for ns in next_states(state):            
            if ns not in Parent:
                Parent[ns] = state
                Stack.append(ns)
                if ns == goal:
                    return iter_path_to(goal, Parent)

## Iterative Tiefensuche (Iterative Deepening)

In [12]:
def depth_limited_search(state, goal, next_states, Path, PathSet, limit):
    if state == goal:
        return Path
    if len(Path) == limit:
        return None
    for ns in next_states(state):
        if ns not in PathSet:
            Path   .append(ns)
            PathSet.add(ns)
            Result = depth_limited_search(ns, goal, next_states, Path, PathSet, limit)
            if Result:
                return Result
            Path   .pop()
            PathSet.remove(ns) # remove this line for faster, but non-optimal solution
    return None

In [13]:
def id_search(start, goal, next_states):
    limit = 32
    while True:
        Path = depth_limited_search(start, goal, next_states, [start], { start }, limit)
        if Path is not None:
            return Path
        limit += 1
        print(f'limit = {limit}')

## Best First Search

In [14]:
def manhattan(stateA, stateB):
    n = len(stateA)
    PositionsB = {}
    for row in range(n):
        for col in range(n): 
            tile = stateB[row][col]
            PositionsB[tile] = (row, col)
    result = 0
    for rowA in range(n):
        for colA in range(n): 
            tile = stateA[rowA][colA]
            if tile != 0:
                rowB, colB = PositionsB[tile]
                result += abs(rowA - rowB) + abs(colA - colB)
    return result

In [15]:
def bfs_search(start, goal, next_states, heuristic):
    PrioQueue = [ (heuristic(start, goal), [start]) ]
    while PrioQueue:
        _, Path = heapq.heappop(PrioQueue)
        state   = Path[-1]
        if state == goal:
            return Path
        for ns in next_states(state):
            if ns not in Path:
                d = heuristic(ns, goal)
                heapq.heappush(PrioQueue, (d, Path + [ns]))

## A*-Suche

In [16]:
def rec_path_to(state, Parent):
    p = Parent[state]
    if p == state:
        return [state]
    return rec_path_to(p, Parent) + [state]

In [17]:
def manhattan(stateA, stateB):
    n = len(stateA)
    PositionsB = {}
    for row in range(n):
        for col in range(n): 
            tile = stateB[row][col]
            PositionsB[tile] = (row, col)
    result = 0
    for rowA in range(n):
        for colA in range(n): 
            tile = stateA[rowA][colA]
            if tile != 0:
                rowB, colB = PositionsB[tile]
                result += abs(rowA - rowB) + abs(colA - colB)
    return result

In [18]:
def a_star_search(start, goal, next_states, heuristic):
    Parent   = { start:start }
    Distance = { start: 0 }           
    estGoal  = heuristic(start, goal)
    Estimate = { start: estGoal }
    Frontier = Set() # Set ist eine Queue-Implementierung von Herr Stroetmann
    Frontier.insert( (estGoal, start) )
    while Frontier:
        _, state = Frontier.pop()
        if state == goal:
            return rec_path_to(state, Parent)
        stateDist = Distance[state]
        for ns in next_states(state):
            oldEstimate = Estimate.get(ns, None)
            newEstimate = stateDist + 1 + heuristic(ns, goal)
            if oldEstimate is None or newEstimate < oldEstimate:
                Distance[ns] = stateDist + 1
                Estimate[ns] = newEstimate
                Parent[ns]   = state
                Frontier.insert( (newEstimate, ns) )
                if oldEstimate is not None:
                    Frontier.delete( (oldEstimate, ns) )

## Iterative Deepening A*-Search

In [19]:
def manhattan(stateA, stateB):
    n = len(stateA)
    PositionsB = {}
    for row in range(n):
        for col in range(n): 
            tile = stateB[row][col]
            PositionsB[tile] = (row, col)
    result = 0
    for rowA in range(n):
        for colA in range(n): 
            tile = stateA[rowA][colA]
            if tile != 0:
                rowB, colB = PositionsB[tile]
                result += abs(rowA - rowB) + abs(colA - colB)
    return result

In [20]:
def dl_search(Path, goal, next_states, limit, heuristic):
    state    = Path[-1]
    distance = len(Path) - 1
    total    = distance + heuristic(state, goal)
    if total > limit:
        return total
    if state == goal:
        return Path
    smallest = float("Inf")  # infinity
    for ns in next_states(state):
        if ns not in Path:
            Solution = dl_search(Path + [ns], goal, next_states, limit, heuristic)
            if isinstance(Solution, list):
                return Solution
            smallest = min(smallest, Solution)
    return smallest

In [21]:
def ida_search(start, goal, next_states, heuristic):
    limit = heuristic(start, goal)
    while True:
        print(f'Trying to find a solution of length {limit}.')
        Path = dl_search([start], goal, next_states, limit, heuristic)
        if isinstance(Path, list):
            return Path
        limit = Path

## Beispiel Missionare

$\texttt{problem}(m, i)$ is `True` if there is a problem on a shore that has $m$ missionaries and $i$ infidels.
For a problem to arise, the number $m$ of missionaries needs to be greater than $0$ but less than the number $i$ of
infidels.

In [22]:
def problem(m, i): 
    return 0 < m < i

def no_problem(m, i): 
    return not problem(m, i) and not problem(3 - m, 3 - i)

In [23]:
def next_states_mis(state):
    m, i, b = state
    if  b == 1:
        return { (m-mb, i-ib, 0) for mb in range(m+1)
                                 for ib in range(i+1)
                                 if 1 <= mb + ib <= 2 and no_problem(m-mb, i-ib) 
               }
    else:
        return { (m+mb, i+ib, 1) for mb in range(3-m+1)
                                 for ib in range(3-i+1)
                                 if 1 <= mb + ib <= 2 and no_problem(m+mb, i+ib) 
               }

Initially, all missionaries, all infidels and the boat are on the left shore.
The goal is to have everybody on the right shore, hence the numbers on the left shore
should all be $0$.

In [24]:
start = (3, 3, 1)
goal  = (0, 0, 0)

In [25]:
def fillCharsLeft(x, n):
    s = str(x)
    m = n - len(s)
    return m * " " + s

def fillCharsRight(x, n):
    s = str(x)
    m = n - len(s)
    return s + m * " "

def fillCharsBoth(x, n):
    s  = str(x)
    ml = (n     - len(s)) // 2
    mr = (n + 1 - len(s)) // 2
    return ml * " " + s + mr * " "

def printState(m, k, b):
     print( fillCharsRight(m * "M", 6) + 
            fillCharsRight(k * "K", 6) + 
            fillCharsRight(b * "B", 3) + "    |~~~~~|    " + 
            fillCharsLeft((3 - m) * "M", 6) + 
            fillCharsLeft((3 - k) * "K", 6) + 
            fillCharsLeft((1 - b) * "B", 3) 
          )

def printBoat(m1, k1, b1, m2, k2, b2):
    if b1 == 1:
        if m1 < m2:
            print("Error in printBoat: negative number of missionaries in the boat!")
            return
        if k1 < k2:
            print("Error in printBoat: negative number of infidels in the boat!")
            return
        print(19*" " + "> " + fillCharsBoth((m1-m2)*"M" + " " + (k1-k2)*"K", 3) + " >")
    else:
        if m1 > m2:
            print("Error in printBoat: negative number of missionaries in the boat!")
            return
        if k1 > k2:
            print("Error in printBoat: negative number of infidels in the boat!")
            return
        print(19*" " + "< " + fillCharsBoth((m2-m1)*"M" + " " + (k2-k1)*"K", 3) + " <")

def printPath(Path):
    print("Solution:\n")
    for i in range(len(Path) - 1):
        m1, k1, b1 = Path[i]
        m2, k2, b2 = Path[i+1]
        printState(m1, k1, b1)
        printBoat(m1, k1, b1, m2, k2, b2)
    m, k, b = Path[-1]
    printState(m, k, b)

In [26]:
%%time
%memit Path = bf_search(start, goal, next_states_mis)

number of states:  15
peak memory: 77.11 MiB, increment: 0.14 MiB
CPU times: total: 62.5 ms
Wall time: 928 ms


In [27]:
%%time
%memit Path = bbf_search(start, goal, next_states_mis)

peak memory: 77.15 MiB, increment: 0.02 MiB
CPU times: total: 31.2 ms
Wall time: 889 ms


In [28]:
%%time
%memit Path = df_search(start, goal, next_states_mis)

peak memory: 77.16 MiB, increment: 0.01 MiB
CPU times: total: 46.9 ms
Wall time: 876 ms


In [29]:
%%time
%memit Path = id_search(start, goal, next_states_mis)

peak memory: 77.16 MiB, increment: 0.00 MiB
CPU times: total: 31.2 ms
Wall time: 952 ms


## Beispiel Sliding Puzzle

### Animation

The package `ipycanvas`, which is imported below, can be installed using the following command:
```
    conda install -c conda-forge ipycanvas
```
This package is useful for drawings and animations.  Its documentation can be found at:
  https://ipycanvas.readthedocs.io/en/latest/.

In [30]:
import ipycanvas as cnv

The module `time` is part of the standard library, so it is preinstalled.  We have imported it because we need the function `time.sleep(secs)` to pause the animation for a specified time.

In [31]:
import time

The global variable `Colors` specifies the colors of the tiles.

In [32]:
Colors = ['white', 'lightblue', 'pink', 'magenta', 'orange', 'red', 'yellow', 'lightgreen', 'gold',
          'CornFlowerBlue', 'Coral', 'Cyan', 'orchid', 'DarkSalmon', 'DeepPink', 'green'
         ] 

The global variable `size` specifies the size of one tile in pixels.

In [33]:
size = 100

The function `draw(State, canvas, dx, dy, tile, x)` draws a given `State` of the sliding puzzle, where `tile` has been moved by `offset` pixels into the direction `(dx, dy)`.

In [34]:
def draw(State, canvas, dx, dy, tile, offset):
    canvas.text_align    = 'center'
    canvas.text_baseline = 'middle'
    with cnv.hold_canvas(canvas):
        canvas.clear()
        n = len(State)
        for row in range(n):
            for col in range(n):
                tile_to_draw = State[row][col]
                color = Colors[tile_to_draw]
                canvas.fill_style = color
                if tile_to_draw not in (0, tile):
                    x = col * size
                    y = row * size
                    canvas.fill_rect(x, y, size, size)
                    canvas.line_width = 3.0
                    x += size // 2
                    y += size // 2
                    canvas.stroke_text(str(tile_to_draw), x, y)
                elif tile_to_draw == tile:
                    x = col * size + offset * dx
                    y = row * size + offset * dy
                    canvas.fill_rect(x, y, size, size)
                    canvas.line_width = 3.0
                    x += size // 2
                    y += size // 2
                    if tile_to_draw != 0:
                        canvas.stroke_text(str(tile_to_draw), x, y)

In [35]:
def create_canvas(n): 
    canvas = cnv.Canvas(size=(size * n, size * n))
    canvas.font = '100px serif'
    return canvas

The global variable `delay` controls the speed of the animation.

In [36]:
delay = 0.0005

The function call `tile_and_direction(state, next_state)` takes a state and the state that follows this state and returns a triple `(tile, dx, dy)` where `tile` is the tile that is moved to transform `state` into `next_state` and `(dx, dy)` is the direction in which this tile is moved.

In [37]:
def tile_and_direction(state, next_state):
    row0, col0 = find_tile(0, state)
    row1, col1 = find_tile(0, next_state)
    return state[row1][col1], col0-col1, row0-row1

Given a list of states representing a solution to the sliding puzzle, the function call 
`animation(Solution)` animates the solution.

In [38]:
def animation(Solution):
    start = Solution[0]
    n = len(start)
    canvas = create_canvas(n)
    draw(start, canvas, 0, 0, 0, 0)
    m = len(Solution)
    display(canvas)
    for i in range(m-1):
        state = Solution[i]
        tile, dx, dy = tile_and_direction(state, Solution[i+1])
        for offset in range(size+1):
            draw(state, canvas, dx, dy, tile, offset)
            time.sleep(delay)

### Code

In [39]:
def move_dir(State, row, col, dx, dy):
    State = to_list(State)
    State[row     ][col     ] = State[row + dx][col + dy]
    State[row + dx][col + dy] = 0
    return to_tuple(State)

In [40]:
def find_tile(tile, State):
    n = len(State)
    for row in range(n):
        for col in range(n):
            if State[row][col] == tile:
                return row, col

In [41]:
def next_states_sliding(State):
    n          = len(State)
    row, col   = find_tile(0, State)
    New_States = set()
    Directions = [ (1, 0), (-1, 0), (0, 1), (0, -1) ]
    for dx, dy in Directions:
        if row + dx in range(n) and col + dy in range(n):
            New_States.add(move_dir(State, row, col, dx, dy))
    return New_States

In [42]:
start = ( (8, 0, 6),
          (5, 4, 7),
          (2, 3, 1)
        )
goal = ( (0, 1, 2), 
         (3, 4, 5), 
         (6, 7, 8)
       )

In [43]:
%%time
%memit Path = bf_search(start, goal, next_states_sliding)

number of states:  181440
peak memory: 152.88 MiB, increment: 63.54 MiB
CPU times: total: 2.14 s
Wall time: 3.03 s


In [44]:
%%time
%memit Path = bbf_search(start, goal, next_states_sliding)

peak memory: 105.98 MiB, increment: 1.48 MiB
CPU times: total: 125 ms
Wall time: 975 ms


In [45]:
%%time
%memit Path = df_search(start, goal, next_states_sliding)

peak memory: 103.84 MiB, increment: 3.16 MiB
CPU times: total: 938 ms
Wall time: 1.79 s


In [46]:
%%time
%memit Path = id_search(start, goal, next_states_sliding)

peak memory: 102.59 MiB, increment: 0.01 MiB
CPU times: total: 5.45 s
Wall time: 6.29 s


In [47]:
%%time
%memit Path = bfs_search(start, goal, next_states_sliding, manhattan)

peak memory: 99.63 MiB, increment: 0.04 MiB
CPU times: total: 46.9 ms
Wall time: 881 ms


In [48]:
%%time
%memit Path = a_star_search(start, goal, next_states_sliding, manhattan)

peak memory: 100.52 MiB, increment: 0.89 MiB
CPU times: total: 406 ms
Wall time: 1.23 s


In [49]:
%%time
%memit Path = ida_search(start, goal, next_states_sliding, manhattan)

Trying to find a solution of length 21.
Trying to find a solution of length 23.
Trying to find a solution of length 25.
Trying to find a solution of length 27.
Trying to find a solution of length 29.
Trying to find a solution of length 31.
peak memory: 100.52 MiB, increment: 0.00 MiB
CPU times: total: 359 ms
Wall time: 1.18 s


In [50]:
#animation(Path)